# Contrastes de hipótesis mediante estadísticos de la imagen

Crear función con la que aplicar contrates de hipótesis de normalidad, homocedasticidad y comparación entre grupos

In [1]:
from os import listdir
lista_img_chl = listdir('/content/chl_dataset')
lista_img_sst = listdir('/content/sst_dataset')
lista_img_poc = listdir('/content/poc_dataset')

In [2]:
len(lista_img_chl)
#len(lista_img_sst)
#len(lista_img_poc)

271

In [3]:
#lista_img_poc.remove(".ipynb_checkpoints")
len(lista_img_poc)

271

In [4]:
#Intro
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.stats import skew, kurtosis

def extraer_estadisticos(img, nombre_img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#canal de grises
    img = img.astype(np.float32) / 255.0#normalizar al rango [0,1]

    altura, ancho = img.shape
    tercio = ancho // 3#dividir la imagen en 3 para obtener caracteristicas locales izq, cent y der
    #Dividr la imagen en 3 zonas
    zona_izq = img[:, :tercio]
    zona_cent = img[:, tercio:2*tercio]
    zona_der = img[:, 2*tercio:]

    return {
        "img": nombre_img,
        "media_global": np.mean(img),
        "std_global": np.std(img),
        "max_global": np.max(img),
        "min_global": np.min(img),
        "mediana_global": np.median(img),
        "skew_global": skew(img.flatten()),
        "kurtosis_global": kurtosis(img.flatten()),
        "media_izq": np.mean(zona_izq),
        "media_cent": np.mean(zona_cent),
        "media_der": np.mean(zona_der),
        "std_izq": np.std(zona_izq),
        "std_cent": np.std(zona_cent),
        "std_der": np.std(zona_der),
    }

def dataframe_estadisticos(lista_img, img_path):
    registros = []

    for nombre_img in lista_img:
        img = cv2.imread(img_path + nombre_img)

        if img is None:
            print(f"No se pudo cargar la imagen: {nombre_img}")
            continue

        stats = extraer_estadisticos(img, nombre_img)
        registros.append(stats)

    return pd.DataFrame(registros)

In [5]:
def ordenar_lista_cronologicamente(lista_img):
    return sorted(lista_img, key=lambda x: datetime.strptime(x.split('.')[1][:6], "%Y%m"))

lista_img_chl_ordenado = ordenar_lista_cronologicamente(lista_img_chl)
lista_img_poc_ordenado = ordenar_lista_cronologicamente(lista_img_poc)
lista_img_sst_ordenado = ordenar_lista_cronologicamente(lista_img_sst)

fechas_chl = [f.split('.')[1][:6] for f in lista_img_chl_ordenado]
fechas_poc = [f.split('.')[1][:6] for f in lista_img_poc_ordenado]
fechas_sst = [f.split('.')[1][:6] for f in lista_img_sst_ordenado]


dataset_chl = dataframe_estadisticos(lista_img_chl_ordenado, 'chl_dataset/')
dataset_chl["fecha"] = fechas_chl
dataset_chl = dataset_chl.rename(columns={col: col + '_chl' for col in dataset_chl.columns if col != 'fecha'})

dataset_poc = dataframe_estadisticos(lista_img_poc_ordenado, 'poc_dataset/')
dataset_poc["fecha"] = fechas_poc
dataset_poc = dataset_poc.rename(columns={col: col + '_poc' for col in dataset_poc.columns if col != 'fecha'})

dataset_sst = dataframe_estadisticos(lista_img_sst_ordenado, 'sst_dataset/')
dataset_sst["fecha"] = fechas_sst
dataset_sst = dataset_sst.rename(columns={col: col + '_sst' for col in dataset_sst.columns if col != 'fecha'})

In [6]:
dataset_estadisticos = dataset_chl.merge(dataset_poc, on="fecha").merge(dataset_sst, on="fecha")
dataset_estadisticos

,img_chl,media_global_chl,std_global_chl,max_global_chl,min_global_chl,mediana_global_chl,skew_global_chl,kurtosis_global_chl,media_izq_chl,media_cent_chl,...,min_global_sst,mediana_global_sst,skew_global_sst,kurtosis_global_sst,media_izq_sst,media_cent_sst,media_der_sst,std_izq_sst,std_cent_sst,std_der_sst
0,AQUA_MODIS.20020801_20020831.L3m.MO.CHL.x_chlo...,0.644246,0.252995,0.886275,0.0,0.721569,-1.798700,2.078758,0.755637,0.635294,...,0.0,0.482353,-1.575434,1.553779,0.572357,0.436310,0.310609,0.060581,0.163471,0.172562
1,AQUA_MODIS.20020901_20020930.L3m.MO.CHL.x_chlo...,0.653256,0.257046,0.886275,0.0,0.729412,-1.717000,1.845603,0.798109,0.639653,...,0.0,0.521569,-1.740386,1.857985,0.565809,0.457370,0.329552,0.068671,0.173596,0.182275
2,AQUA_MODIS.20021001_20021031.L3m.MO.CHL.x_chlo...,0.681664,0.251049,0.886275,0.0,0.764706,-2.081497,2.988384,0.772672,0.716211,...,0.0,0.533333,-2.135174,3.081862,0.555287,0.481898,0.357143,0.056309,0.162519,0.197851
3,AQUA_MODIS.20021101_20021130.L3m.MO.CHL.x_chlo...,0.677994,0.250715,0.886275,0.0,0.776471,-2.035709,2.824072,0.785504,0.695938,...,0.0,0.552941,-2.308019,3.454967,0.569625,0.492560,0.405917,0.054567,0.175049,0.223854
4,AQUA_MODIS.20021201_20021231.L3m.MO.CHL.x_chlo...,0.673279,0.240186,0.886275,0.0,0.752941,-2.227126,3.590875,0.731285,0.712552,...,0.0,0.584314,-2.482342,4.232481,0.598267,0.534524,0.448582,0.057306,0.177046,0.237435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,AQUA_MODIS.20241001_20241031.L3m.MO.CHL.x_chlo...,0.614869,0.261634,0.886275,0.0,0.701961,-1.239408,0.616517,0.663130,0.608088,...,0.0,0.470588,-2.233762,3.712403,0.502626,0.441597,0.344275,0.052261,0.147928,0.178956
267,AQUA_MODIS.20241101_20241130.L3m.MO.CHL.x_chlo...,0.447911,0.228027,0.886275,0.0,0.454902,-0.294409,-0.336382,0.542647,0.416387,...,0.0,0.603922,-2.492428,4.403334,0.622356,0.546621,0.456093,0.060036,0.181307,0.232933
268,AQUA_MODIS.20241201_20241231.L3m.MO.CHL.x_chlo...,0.592665,0.265910,0.886275,0.0,0.666667,-1.096294,0.151880,0.728309,0.573529,...,0.0,0.627451,-2.612711,4.949227,0.632896,0.577206,0.481180,0.061213,0.181126,0.242004
269,AQUA_MODIS.20250101_20250131.L3m.MO.CHL.x_chlo...,0.527311,0.281613,0.886275,0.0,0.552941,-0.513188,-0.852228,0.668032,0.466106,...,0.0,0.643137,-2.468160,4.138798,0.651908,0.581513,0.488813,0.061984,0.192194,0.262252


In [9]:
#!pip install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 6.0 MB/s eta 0:00:00


In [10]:
from scipy.stats import shapiro, levene, f_oneway, mannwhitneyu
import pingouin as pg

def analizar_contraste(df, variable, fase_col='fases'):
    print(f"\nVariable: {variable}")

    normales = []
    for fase in df[fase_col].unique():
        grupo = df[df[fase_col] == fase][variable]
        stat, p = shapiro(grupo)
        normales.append(p > 0.05)
        print(f" - Normalidad en {fase}: p-valor = {p:.4f}")

    normalidad = all(normales)

    if normalidad:
        grupo1 = df[df[fase_col] == df[fase_col].unique()[0]][variable]
        grupo2 = df[df[fase_col] == df[fase_col].unique()[1]][variable]
        stat, p_levene = levene(grupo1, grupo2)
        print(f" - Homocedasticidad (Levene): p-valor = {p_levene:.4f}")

        if p_levene > 0.05:
            stat, p_anova = f_oneway(grupo1, grupo2)
            print(f"ANOVA clásico: p-valor = {p_anova:.4f}")
        else:
            resultado = pg.welch_anova(dv=variable, between=fase_col, data=df)
            print(f"Welch: p-valor = {resultado['p-unc'].values[0]:.4f}")

    else:
        grupo1 = df[df[fase_col] == df[fase_col].unique()[0]][variable]
        grupo2 = df[df[fase_col] == df[fase_col].unique()[1]][variable]
        stat, p_mw = mannwhitneyu(grupo1, grupo2, alternative='two-sided')
        print(f"Mann-Whitney U: p-valor = {p_mw:.4f}")

Contraste de hipótesis en Enero: El Niño vs La Niña

In [5]:
dataset_estadisticos_ene = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "01"].reset_index(drop=True)
dataset_estadisticos_ene

dataset_estadisticos_ene["fases"] = ["niño", "neutra", "niño", "niña", "niño", "niña", "niña", "niño",
                                      "niña", "niña", "neutra", "neutra", "niño", "niño", "neutra",
                                      "niña", "niño", "neutra", "niña", "niña", "niña", "niño", "neutra"]

dataset_estadisticos_ene = dataset_estadisticos_ene[dataset_estadisticos_ene["fases"] != "neutra"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_ene, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.0608
 - Normalidad en niña: p-valor = 0.3781
 - Homocedasticidad (Levene): p-valor = 0.2440
ANOVA clásico: p-valor = 0.0793

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.0004
 - Normalidad en niña: p-valor = 0.4450
Mann-Whitney U: p-valor = 0.7430

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0000
 - Normalidad en niña: p-valor = 0.0000
Mann-Whitney U: p-valor = 1.0000

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.0525
 - Normalidad en niña: p-valor = 0.1836
 - Homocedasticidad (Levene): p-valor = 0.2123
ANOVA clásico: p-valor = 0.1791

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.0132
 - Normalidad en niña: p-valor = 0.9795
Mann-Whitney U: p-valor = 0.0055

Variable: kurtosis_global_chl
 

Contraste de hipótesis en Febrero: El Niño vs La Niña

In [6]:
dataset_estadisticos_feb = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "02"].reset_index(drop=True)
dataset_estadisticos_feb

dataset_estadisticos_feb["fases"] = ["niño", "neutra", "niño", "niña", "neutra", "niña", "niña", "niño",
                                      "niña", "niña", "neutra", "neutra", "niño", "niño", "neutra",
                                      "niña", "niño", "neutra", "niña", "niña", "neutra", "niño", "neutra"]

dataset_estadisticos_feb = dataset_estadisticos_feb[dataset_estadisticos_feb["fases"] != "neutra"]



for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_feb, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.3268
 - Normalidad en niña: p-valor = 0.9536
 - Homocedasticidad (Levene): p-valor = 0.1208
ANOVA clásico: p-valor = 0.0983

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.0049
 - Normalidad en niña: p-valor = 0.1987
Mann-Whitney U: p-valor = 0.0721

Variable: max_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 0.0000
Mann-Whitney U: p-valor = 0.4227

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.3193
 - Normalidad en niña: p-valor = 0.4206
 - Homocedasticidad (Levene): p-valor = 0.0573
ANOVA clásico: p-valor = 0.1429

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.3992
 - Normalidad en niña: p-valor = 0.5970
 - Homocedasticidad (Levene): p-valor = 0.0860
ANOVA clásico: p-v

Contraste de hipótesis en Marzo: El Niño vs La Niña

In [16]:
dataset_estadisticos_mar = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "03"].reset_index(drop=True)

dataset_estadisticos_mar["fases"] = ["niño", "niña", "niña", "niño", "neutra", "niña", "neutra", "niña",
                                      "niña", "niña", "neutra", "niña", "niño", "neutra", "neutra",
                                      "niño", "neutra", "neutra", "niño", "neutra", "niña", "neutra"]

dataset_estadisticos_mar = dataset_estadisticos_mar[dataset_estadisticos_mar["fases"] != "neutra"]

for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_mar, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.1165
 - Normalidad en niña: p-valor = 0.9311
 - Homocedasticidad (Levene): p-valor = 0.7514
ANOVA clásico: p-valor = 0.9634

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.2178
 - Normalidad en niña: p-valor = 0.4878
 - Homocedasticidad (Levene): p-valor = 0.9219
ANOVA clásico: p-valor = 0.2707

Variable: max_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.2243
 - Normalidad en niña: p-valor = 0.7807
 - Homocedasticidad (Levene): p-valor = 0.6774
ANOVA clásico: p-valor = 0.8705

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.1716
 - Normalidad en niña: p

Contraste de hipótesis en Abril: El Niño vs La Niña

In [7]:
dataset_estadisticos_abr = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "04"].reset_index(drop=True)
dataset_estadisticos_abr

dataset_estadisticos_abr["fases"] = ["neutra", "neutra", "neutra", "neutra", "neutra", "niña", "neutra", "neutra",
                                      "niña", "niña", "neutra", "neutra", "niño", "niño", "neutra",
                                      "niña", "niño", "neutra", "niña", "niña", "neutra", "niño"]

dataset_estadisticos_abr = dataset_estadisticos_abr[dataset_estadisticos_abr["fases"] != "neutra"]



for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_abr, col)



Variable: media_global_chl
 - Normalidad en niña: p-valor = 0.4691
 - Normalidad en niño: p-valor = 0.0470
Mann-Whitney U: p-valor = 0.7619

Variable: std_global_chl
 - Normalidad en niña: p-valor = 0.2192
 - Normalidad en niño: p-valor = 0.2160
 - Homocedasticidad (Levene): p-valor = 0.7971
ANOVA clásico: p-valor = 0.8620

Variable: max_global_chl
 - Normalidad en niña: p-valor = 1.0000
 - Normalidad en niño: p-valor = 0.0012
Mann-Whitney U: p-valor = 0.3074

Variable: min_global_chl
 - Normalidad en niña: p-valor = 1.0000
 - Normalidad en niño: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niña: p-valor = 0.4661
 - Normalidad en niño: p-valor = 0.4293
 - Homocedasticidad (Levene): p-valor = 0.0491
Welch: p-valor = 0.9046

Variable: skew_global_chl
 - Normalidad en niña: p-valor = 0.2681
 - Normalidad en niño: p-valor = 0.2091
 - Homocedasticidad (Levene): p-valor = 0.0847
ANOVA clásico: p-valor = 

Contraste de hipótesis en Mayo: El Niño vs La Niña

In [8]:
dataset_estadisticos_may = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "05"].reset_index(drop=True)
dataset_estadisticos_may

dataset_estadisticos_may["fases"] = ["neutra", "neutra", "neutra", "neutra", "neutra", "niña", "neutra", "neutra",
                                      "niña", "neutra", "neutra", "neutra", "niño", "neutra", "neutra",
                                      "neutra", "niño", "neutra", "niña", "niña", "niño", "neutra"]

dataset_estadisticos_may = dataset_estadisticos_may[dataset_estadisticos_may["fases"] != "neutra"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_may, col)


Variable: media_global_chl
 - Normalidad en niña: p-valor = 0.0953
 - Normalidad en niño: p-valor = 0.4850
 - Homocedasticidad (Levene): p-valor = 0.8494
ANOVA clásico: p-valor = 0.1372

Variable: std_global_chl
 - Normalidad en niña: p-valor = 0.0749
 - Normalidad en niño: p-valor = 0.0333
Mann-Whitney U: p-valor = 0.4000

Variable: max_global_chl
 - Normalidad en niña: p-valor = 0.0012
 - Normalidad en niño: p-valor = 1.0000
Mann-Whitney U: p-valor = 0.5637

Variable: min_global_chl
 - Normalidad en niña: p-valor = 1.0000
 - Normalidad en niño: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niña: p-valor = 0.0850
 - Normalidad en niño: p-valor = 0.7804
 - Homocedasticidad (Levene): p-valor = 0.7795
ANOVA clásico: p-valor = 0.1378

Variable: skew_global_chl
 - Normalidad en niña: p-valor = 0.0534
 - Normalidad en niño: p-valor = 0.0423
Mann-Whitney U: p-valor = 0.1143

Variable: kurtosis_global_chl
 

Contraste de hipótesis en Junio: El Niño vs La Niña

In [9]:
dataset_estadisticos_jun = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "06"].reset_index(drop=True)
dataset_estadisticos_jun

dataset_estadisticos_jun["fases"] = ["neutra", "neutra", "neutra", "neutra", "niña", "niña", "neutra",
                                      "niña", "neutra", "neutra", "neutra", "neutra", "niño", "neutra",
                                      "neutra", "neutra", "niño", "neutra", "neutra", "niña", "niño","neutra"]

dataset_estadisticos_jun = dataset_estadisticos_jun[dataset_estadisticos_jun["fases"] != "neutra"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_jun, col)


Variable: media_global_chl
 - Normalidad en niña: p-valor = 0.7798
 - Normalidad en niño: p-valor = 0.8867
 - Homocedasticidad (Levene): p-valor = 0.0511
ANOVA clásico: p-valor = 0.2041

Variable: std_global_chl
 - Normalidad en niña: p-valor = 0.3353
 - Normalidad en niño: p-valor = 0.9649
 - Homocedasticidad (Levene): p-valor = 0.2403
ANOVA clásico: p-valor = 0.2206

Variable: max_global_chl
 - Normalidad en niña: p-valor = 1.0000
 - Normalidad en niño: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: min_global_chl
 - Normalidad en niña: p-valor = 1.0000
 - Normalidad en niño: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niña: p-valor = 0.7803
 - Normalidad en niño: p-valor = 0.8168
 - Homocedasticidad (Levene): p-valor = 0.0350
Welch: p-valor = 0.1391

Variable: skew_global_chl
 - Normalidad en niña: p-valor = 0.4920
 - Normalidad en niño: p-valor =

Contraste de hipótesis en Julio: El Niño vs La Niña

In [10]:
dataset_estadisticos_jul = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "07"].reset_index(drop=True)
dataset_estadisticos_jul

dataset_estadisticos_jul["fases"] = ["neutra", "niño", "neutra", "neutra", "niña", "neutra", "niño",
                                      "niña", "niña", "neutra", "neutra", "neutra", "niño", "neutra",
                                      "neutra", "neutra", "neutra", "neutra", "neutra", "niña", "niño","neutra"]

dataset_estadisticos_jul = dataset_estadisticos_jul[dataset_estadisticos_jul["fases"] != "neutra"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_jul, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.5551
 - Normalidad en niña: p-valor = 0.1336
 - Homocedasticidad (Levene): p-valor = 0.9295
ANOVA clásico: p-valor = 0.2186

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.7760
 - Normalidad en niña: p-valor = 0.7213
 - Homocedasticidad (Levene): p-valor = 0.5017
ANOVA clásico: p-valor = 0.2461

Variable: max_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.8025
 - Normalidad en niña: p-valor = 0.6500
 - Homocedasticidad (Levene): p-valor = 0.6200
ANOVA clásico: p-valor = 0.2142

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.6132
 - Normalidad en niña: p

Contraste de hipótesis en Agosto: El Niño vs La Niña

In [11]:
dataset_estadisticos_ago = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "08"].reset_index(drop=True)
dataset_estadisticos_ago

dataset_estadisticos_ago["fases"] = ["niño", "neutra", "niño", "neutra", "neutra", "niña", "neutra", "niño",
                                      "niña", "niña", "neutra", "neutra", "neutra", "niño", "niña",
                                      "neutra", "neutra", "neutra", "niña", "niña", "niña", "niño","neutra"]

dataset_estadisticos_ago = dataset_estadisticos_ago[dataset_estadisticos_ago["fases"] != "neutra"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_ago, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.5204
 - Normalidad en niña: p-valor = 0.0437
Mann-Whitney U: p-valor = 0.8763

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.2193
 - Normalidad en niña: p-valor = 0.8886
 - Homocedasticidad (Levene): p-valor = 0.1921
ANOVA clásico: p-valor = 0.5908

Variable: max_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.7038
 - Normalidad en niña: p-valor = 0.1465
 - Homocedasticidad (Levene): p-valor = 0.9835
ANOVA clásico: p-valor = 0.8107

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.4906
 - Normalidad en niña: p-valor = 0.0093
Mann-Whitney U: p-valor = 0.63

Contraste de hipótesis en Septiembre: El Niño vs La Niña

In [12]:
dataset_estadisticos_sep = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "09"].reset_index(drop=True)
dataset_estadisticos_sep

dataset_estadisticos_sep["fases"] = ["niño", "neutra", "niño", "neutra", "niño", "niña", "neutra", "niño",
                                      "niña", "niña", "neutra", "neutra", "neutra", "niño", "niña",
                                      "neutra", "niño", "neutra", "niña", "niña", "niña", "niño","neutra"]

dataset_estadisticos_sep = dataset_estadisticos_sep[dataset_estadisticos_sep["fases"] != "neutra"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_sep, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.0143
 - Normalidad en niña: p-valor = 0.0959
Mann-Whitney U: p-valor = 0.1282

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.0012
 - Normalidad en niña: p-valor = 0.5563
Mann-Whitney U: p-valor = 0.8048

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0000
 - Normalidad en niña: p-valor = 1.0000
Mann-Whitney U: p-valor = 0.3914

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.0480
 - Normalidad en niña: p-valor = 0.0821
Mann-Whitney U: p-valor = 0.1329

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.1032
 - Normalidad en niña: p-valor = 0.0012
Mann-Whitney U: p-valor = 0.2593

Variable: kurtosis_global_chl
 - Normalidad en niño: p-valor = 0.0545
 - Normalidad en niña: p-valor = 0.0040
Mann-Whitney 

Contraste de hipótesis en Octubre: El Niño vs La Niña

In [13]:
dataset_estadisticos_oct = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "10"].reset_index(drop=True)
dataset_estadisticos_oct

dataset_estadisticos_oct["fases"] = ["niño", "neutra", "niño", "neutra", "niño", "niña", "neutra", "niño",
                                      "niña", "niña", "neutra", "neutra", "niño", "niño", "niña",
                                      "niña", "niño", "neutra", "niña", "niña", "niña", "niño","neutra"]

dataset_estadisticos_oct = dataset_estadisticos_oct[dataset_estadisticos_oct["fases"] != "neutra"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_oct, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.0326
 - Normalidad en niña: p-valor = 0.1841
Mann-Whitney U: p-valor = 0.7209

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.0240
 - Normalidad en niña: p-valor = 0.2419
Mann-Whitney U: p-valor = 0.7209

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0000
 - Normalidad en niña: p-valor = 0.0000
Mann-Whitney U: p-valor = 0.5897

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.0462
 - Normalidad en niña: p-valor = 0.3549
Mann-Whitney U: p-valor = 0.5280

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.7091
 - Normalidad en niña: p-valor = 0.3481
 - Homocedasticidad (Levene): p-valor = 0.0585
ANOVA clásico: p-valor = 0.0274

Variable: kurtosis_global_chl
 - Normalidad en niño: p-valor = 0.6497
 - Norm

Contraste de hipótesis en Noviembre: El Niño vs La Niña

In [15]:
dataset_estadisticos_nov = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "11"].reset_index(drop=True)

dataset_estadisticos_nov["fases"] = ["niño", "neutra", "niño", "niña", "niño", "niña",
                                      "niña", "niño", "niña", "niña", "neutra", "neutra",
                                      "niño", "niño", "niña", "niña", "niño", "neutra",
                                      "niña", "niña", "niña", "niño", "neutra"]

dataset_estadisticos_nov = dataset_estadisticos_nov[dataset_estadisticos_nov["fases"] != "neutra"]

for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_nov, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.1188
 - Normalidad en niña: p-valor = 0.1298
 - Homocedasticidad (Levene): p-valor = 0.0681
ANOVA clásico: p-valor = 0.2335

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.0085
 - Normalidad en niña: p-valor = 0.0942
Mann-Whitney U: p-valor = 0.2370

Variable: max_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.0728
 - Normalidad en niña: p-valor = 0.0629
 - Homocedasticidad (Levene): p-valor = 0.1371
ANOVA clásico: p-valor = 0.3617

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.2338
 - Normalidad en niña: p-valor = 0.4179
 - Homocedasticidad (Levene): 

Contraste de hipótesis en Diciembre: El Niño vs La Niña

In [14]:
dataset_estadisticos_dic = dataset_estadisticos[dataset_estadisticos["img_chl"].str[15:17] == "12"].reset_index(drop=True)
dataset_estadisticos_dic

dataset_estadisticos_dic["fases"] = ["niño", "neutra", "niño", "niña", "niño", "niña",
                                      "niña", "niño", "niña", "niña", "neutra", "neutra",
                                      "niño", "niño", "niña", "niña", "niño", "neutra",
                                      "niña", "niña", "niña", "niño", "neutra"]

dataset_estadisticos_dic = dataset_estadisticos_dic[dataset_estadisticos_dic["fases"] != "neutra"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_dic, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.0043
 - Normalidad en niña: p-valor = 0.0023
Mann-Whitney U: p-valor = 0.4598

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.7504
 - Normalidad en niña: p-valor = 0.7768
 - Homocedasticidad (Levene): p-valor = 0.0095
Welch: p-valor = 0.0326

Variable: max_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.0133
 - Normalidad en niña: p-valor = 0.0029
/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local

# Contrastes de hipótesis mediante estadísticos de la imagen

In [11]:
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.stats import skew, kurtosis

def extraer_estadisticos_region(img, nombre_img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype(np.float32) / 255.0

    # Coordenadas del rectángulo de interés (y: 2 a 12, x: 17 a 31)
    x_ini, x_fin = 17, 31  # ancho = 14
    y_ini, y_fin = 2, 12   # alto = 10

    region = img[y_ini:y_fin, x_ini:x_fin]

    # División en tercios verticales
    altura, ancho = region.shape
    tercio = ancho // 3

    zona_izq = region[:, :tercio]
    zona_cent = region[:, tercio:2*tercio]
    zona_der = region[:, 2*tercio:]

    return {
        "img": nombre_img,
        "media_global": np.mean(region),
        "std_global": np.std(region),
        "max_global": np.max(region),
        "min_global": np.min(region),
        "mediana_global": np.median(region),
        "skew_global": skew(region.flatten()),
        "kurtosis_global": kurtosis(region.flatten()),
        "media_izq": np.mean(zona_izq),
        "media_cent": np.mean(zona_cent),
        "media_der": np.mean(zona_der),
        "std_izq": np.std(zona_izq),
        "std_cent": np.std(zona_cent),
        "std_der": np.std(zona_der),
    }

def dataframe_estadisticos_region(lista_img, img_path):
    registros = []

    for nombre_img in lista_img:
        img = cv2.imread(img_path + nombre_img)

        if img is None:
            print(f"No se pudo cargar la imagen: {nombre_img}")
            continue

        stats = extraer_estadisticos_region(img, nombre_img)
        registros.append(stats)

    return pd.DataFrame(registros)


In [12]:
def ordenar_lista_cronologicamente(lista_img):
    return sorted(lista_img, key=lambda x: datetime.strptime(x.split('.')[1][:6], "%Y%m"))

lista_img_chl_ordenado_eddy = ordenar_lista_cronologicamente(lista_img_chl)
lista_img_poc_ordenado_eddy = ordenar_lista_cronologicamente(lista_img_poc)
lista_img_sst_ordenado_eddy = ordenar_lista_cronologicamente(lista_img_sst)

fechas_chl_eddy = [f.split('.')[1][:6] for f in lista_img_chl_ordenado_eddy]
fechas_poc_eddy = [f.split('.')[1][:6] for f in lista_img_poc_ordenado_eddy]
fechas_sst_eddy = [f.split('.')[1][:6] for f in lista_img_sst_ordenado_eddy]


dataset_chl_eddy = dataframe_estadisticos_region(lista_img_chl_ordenado_eddy, 'chl_dataset/')
dataset_chl_eddy["fecha"] = fechas_chl_eddy
dataset_chl_eddy = dataset_chl_eddy.rename(columns={col: col + '_chl' for col in dataset_chl_eddy.columns if col != 'fecha'})

dataset_poc_eddy = dataframe_estadisticos_region(lista_img_poc_ordenado_eddy, 'poc_dataset/')
dataset_poc_eddy["fecha"] = fechas_poc_eddy
dataset_poc_eddy = dataset_poc_eddy.rename(columns={col: col + '_poc' for col in dataset_poc_eddy.columns if col != 'fecha'})

dataset_sst_eddy = dataframe_estadisticos_region(lista_img_sst_ordenado_eddy, 'sst_dataset/')
dataset_sst_eddy["fecha"] = fechas_sst_eddy
dataset_sst_eddy = dataset_sst_eddy.rename(columns={col: col + '_sst' for col in dataset_sst_eddy.columns if col != 'fecha'})

In [13]:
dataset_estadisticos_eddy = dataset_chl_eddy.merge(dataset_poc_eddy, on="fecha").merge(dataset_sst_eddy, on="fecha")
dataset_estadisticos_eddy

,img_chl,media_global_chl,std_global_chl,max_global_chl,min_global_chl,mediana_global_chl,skew_global_chl,kurtosis_global_chl,media_izq_chl,media_cent_chl,...,min_global_sst,mediana_global_sst,skew_global_sst,kurtosis_global_sst,media_izq_sst,media_cent_sst,media_der_sst,std_izq_sst,std_cent_sst,std_der_sst
0,AQUA_MODIS.20020801_20020831.L3m.MO.CHL.x_chlo...,0.657059,0.163642,0.839216,0.0,0.701961,-2.735421,8.376847,0.626569,0.666863,...,0.0,0.458824,-3.818824,17.685535,0.527255,0.485392,0.418824,0.035737,0.029923,0.112802
1,AQUA_MODIS.20020901_20020930.L3m.MO.CHL.x_chlo...,0.699384,0.119966,0.878431,0.0,0.721569,-4.262573,22.069414,0.677059,0.731078,...,0.0,0.521569,-4.288606,22.815432,0.552255,0.526373,0.460327,0.029890,0.036949,0.109179
2,AQUA_MODIS.20021001_20021031.L3m.MO.CHL.x_chlo...,0.782157,0.147942,0.886275,0.0,0.811765,-4.216003,19.434544,0.802059,0.823431,...,0.0,0.533333,-5.353870,31.230888,0.554902,0.532451,0.476993,0.013837,0.022018,0.111083
3,AQUA_MODIS.20021101_20021130.L3m.MO.CHL.x_chlo...,0.753361,0.127111,0.878431,0.0,0.784314,-4.469247,23.621273,0.755490,0.774216,...,0.0,0.549020,-6.392259,39.830414,0.556863,0.554510,0.518497,0.011862,0.012207,0.119249
4,AQUA_MODIS.20021201_20021231.L3m.MO.CHL.x_chlo...,0.793053,0.122816,0.886275,0.0,0.803922,-5.717978,34.291206,0.799020,0.819020,...,0.0,0.576471,-6.429445,40.170914,0.590784,0.585098,0.547516,0.013286,0.009812,0.125703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,AQUA_MODIS.20241001_20241031.L3m.MO.CHL.x_chlo...,0.649300,0.195286,0.886275,0.0,0.698039,-1.158619,1.024426,0.586667,0.641471,...,0.0,0.450980,-4.485044,25.262228,0.494118,0.474608,0.421307,0.036651,0.033026,0.097271
267,AQUA_MODIS.20241101_20241130.L3m.MO.CHL.x_chlo...,0.386863,0.105830,0.552941,0.0,0.396078,-1.087343,1.970940,0.428922,0.429902,...,0.0,0.592157,-6.284744,38.925388,0.604412,0.599608,0.556928,0.012743,0.015509,0.128353
268,AQUA_MODIS.20241201_20241231.L3m.MO.CHL.x_chlo...,0.564286,0.164423,0.878431,0.0,0.564706,-0.689316,1.526881,0.617941,0.618627,...,0.0,0.627451,-6.499817,40.745697,0.641373,0.632255,0.592222,0.004718,0.006987,0.135991
269,AQUA_MODIS.20250101_20250131.L3m.MO.CHL.x_chlo...,0.415630,0.165282,0.858824,0.0,0.421569,0.143309,1.196328,0.425784,0.455196,...,0.0,0.643137,-6.486337,40.626045,0.651569,0.643039,0.608627,0.009261,0.010864,0.139850


Crear dataset estadísticos de Enero zona eddy

In [ ]:
dataset_estadisticos_ene_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "01"].reset_index(drop=True)

dataset_estadisticos_ene_eddy["fases"] = ["niño", "neutra", "niño", "niña", "niño", "niña", "niña", "niño",
                                      "niña", "niña", "neutra", "neutra", "niño", "niño", "neutra",
                                      "niña", "niño", "neutra", "niña", "niña", "niña", "niño", "neutra"]

dataset_estadisticos_ene_eddy = dataset_estadisticos_ene_eddy[dataset_estadisticos_ene_eddy["fases"] != "neutra"]
dataset_estadisticos_ene_eddy = dataset_estadisticos_ene_eddy.drop(columns = ["fases"])

X_estadisticos_ene_eddy = dataset_estadisticos_ene_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_ene_eddy

dataset_estadisticos_ene_eddy["fases"] = ["niño", "niño", "niña", "niño", "niña", "niña", "niño",
                                      "niña", "niña", "niño", "niño",
                                      "niña", "niño", "niña", "niña", "niña", "niño"]

Contraste de hipótesis en Enero zona eddy: Fase El Niño vs Fase La Niña

In [1]:
for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_ene_eddy, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.0085
 - Normalidad en niña: p-valor = 0.6598
Mann-Whitney U: p-valor = 0.0025

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.9290
 - Normalidad en niña: p-valor = 0.9947
 - Homocedasticidad (Levene): p-valor = 0.9758
ANOVA clásico: p-valor = 0.3205

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0000
 - Normalidad en niña: p-valor = 0.0077
Mann-Whitney U: p-valor = 0.0104

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.0088
 - Normalidad en niña: p-valor = 0.6470
Mann-Whitney U: p-valor = 0.0141

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.9941
 - Normalidad en niña: p-valor = 0.1303
 - Homocedasticidad (Levene): p-valor = 0.5799
ANOVA clásico: p-valor = 0.0002

Variable: kurtosis_global_chl
 

Crear dataset estadísticos de Febrero y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [2]:
dataset_estadisticos_feb_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "02"].reset_index(drop=True)

dataset_estadisticos_feb_eddy["fases"] = ["niño", "neutra", "niño", "niña", "neutra", "niña", "niña", "niño",
                                      "niña", "niña", "neutra", "neutra", "niño", "niño", "neutra",
                                      "niña", "niño", "neutra", "niña", "niña", "neutra", "niño", "neutra"]

dataset_estadisticos_feb_eddy = dataset_estadisticos_feb_eddy[dataset_estadisticos_feb_eddy["fases"] != "neutra"]
dataset_estadisticos_feb_eddy = dataset_estadisticos_feb_eddy.drop(columns = ["fases"])

X_estadisticos_feb_eddy = dataset_estadisticos_feb_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_feb_eddy

dataset_estadisticos_feb_eddy["fases"] = ["niño", "niño", "niña", "niña", "niña", "niño",
                                      "niña", "niña", "niño", "niño",
                                      "niña", "niño", "niña", "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_feb_eddy, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.1302
 - Normalidad en niña: p-valor = 0.9170
 - Homocedasticidad (Levene): p-valor = 0.1634
ANOVA clásico: p-valor = 0.0104

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.9443
 - Normalidad en niña: p-valor = 0.2268
 - Homocedasticidad (Levene): p-valor = 0.7048
ANOVA clásico: p-valor = 0.4350

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0032
 - Normalidad en niña: p-valor = 0.1042
Mann-Whitney U: p-valor = 0.0372

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.0627
 - Normalidad en niña: p-valor = 0.7861
 - Homocedasticidad (Levene): p-valor = 0.1917
ANOVA clásico: p-valor = 0.0116

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.3172
 - Normalidad en niña: p-valor = 0.0590
 - Homocedasticidad

Crear dataset estadísticos de Marzo y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [4]:
dataset_estadisticos_mar_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "03"].reset_index(drop=True)

dataset_estadisticos_mar_eddy["fases"] = ["niño", "niña", "niña", "niño", "neutra", "niña", "neutra", "niña",
                                      "niña", "niña", "neutra", "niña", "niño", "neutra", "neutra",
                                      "niño", "neutra", "neutra", "niño", "neutra", "niña", "neutra"]

dataset_estadisticos_mar_eddy = dataset_estadisticos_mar_eddy[dataset_estadisticos_mar_eddy["fases"] != "neutra"]
dataset_estadisticos_mar_eddy = dataset_estadisticos_mar_eddy.drop(columns = ["fases"])

X_estadisticos_mar_eddy = dataset_estadisticos_mar_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_mar_eddy

dataset_estadisticos_mar_eddy["fases"] = ["niño", "niña", "niña", "niño", "niña", "niña",
                                      "niña", "niña", "niña", "niño",
                                      "niño", "niño", "niña"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_mar_eddy, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.2819
 - Normalidad en niña: p-valor = 0.5496
 - Homocedasticidad (Levene): p-valor = 0.7104
ANOVA clásico: p-valor = 0.9151

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.1151
 - Normalidad en niña: p-valor = 0.2022
 - Homocedasticidad (Levene): p-valor = 0.8700
ANOVA clásico: p-valor = 0.9338

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0228
 - Normalidad en niña: p-valor = 0.0548
Mann-Whitney U: p-valor = 0.9407

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.3470
 - Normalidad en niña: p-valor = 0.6128
 - Homocedasticidad (Levene): p-valor = 0.6607
ANOVA clásico: p-valor = 0.8839

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.4010
 - Normalidad en niña: p-valor = 0.0520
 - Homocedasticidad

Crear dataset estadísticos de Abril y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [3]:
dataset_estadisticos_abr_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "04"].reset_index(drop=True)

dataset_estadisticos_abr_eddy["fases"] = ["neutra", "neutra", "neutra", "neutra", "neutra", "niña", "neutra", "neutra",
                                      "niña", "niña", "neutra", "neutra", "niño", "niño", "neutra",
                                      "niña", "niño", "neutra", "niña", "niña", "neutra", "niño"]

dataset_estadisticos_abr_eddy = dataset_estadisticos_abr_eddy[dataset_estadisticos_abr_eddy["fases"] != "neutra"]
dataset_estadisticos_abr_eddy = dataset_estadisticos_abr_eddy.drop(columns = ["fases"])

X_estadisticos_abr_eddy = dataset_estadisticos_abr_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_abr_eddy

dataset_estadisticos_abr_eddy["fases"] = ["niña",
                                      "niña", "niña", "niño", "niño",
                                      "niña", "niño", "niña", "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_abr_eddy, col)


Variable: media_global_chl
 - Normalidad en niña: p-valor = 0.2766
 - Normalidad en niño: p-valor = 0.1139
 - Homocedasticidad (Levene): p-valor = 0.0678
ANOVA clásico: p-valor = 0.4291

Variable: std_global_chl
 - Normalidad en niña: p-valor = 0.8655
 - Normalidad en niño: p-valor = 0.3943
 - Homocedasticidad (Levene): p-valor = 0.1322
ANOVA clásico: p-valor = 0.9191

Variable: max_global_chl
 - Normalidad en niña: p-valor = 0.3737
 - Normalidad en niño: p-valor = 0.6059
 - Homocedasticidad (Levene): p-valor = 0.4595
ANOVA clásico: p-valor = 0.4701

Variable: min_global_chl
 - Normalidad en niña: p-valor = 1.0000
 - Normalidad en niño: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niña: p-valor = 0.2325
 - Normalidad en niño: p-valor = 0.7628
 - Homocedasticidad (Levene): p-valor = 0.0234
Welch: p-valor = 0.3222

Variable: skew_global_chl
 - Normalidad en niña: p-valor = 0.6611
 - Normalidad en niño

Crear dataset estadísticos de Mayo y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [4]:
dataset_estadisticos_abr_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "05"].reset_index(drop=True)

dataset_estadisticos_abr_eddy["fases"] = ["neutra", "neutra", "neutra", "neutra", "neutra", "niña", "neutra", "neutra",
                                      "niña", "neutra", "neutra", "neutra", "niño", "neutra", "neutra",
                                      "neutra", "niño", "neutra", "niña", "niña", "niño", "neutra"]

dataset_estadisticos_abr_eddy = dataset_estadisticos_abr_eddy[dataset_estadisticos_abr_eddy["fases"] != "neutra"]
dataset_estadisticos_abr_eddy = dataset_estadisticos_abr_eddy.drop(columns = ["fases"])

X_estadisticos_abr_eddy = dataset_estadisticos_abr_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_abr_eddy

dataset_estadisticos_abr_eddy["fases"] = ["niña",
                                      "niña", "niño",
                                      "niño", "niña", "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_abr_eddy, col)


Variable: media_global_chl
 - Normalidad en niña: p-valor = 0.0246
 - Normalidad en niño: p-valor = 0.6314
 Mann-Whitney U: p-valor = 0.2286

Variable: std_global_chl
 - Normalidad en niña: p-valor = 0.5062
 - Normalidad en niño: p-valor = 0.9494
 - Homocedasticidad (Levene): p-valor = 0.5595
 ANOVA clásico: p-valor = 0.0968

Variable: max_global_chl
 - Normalidad en niña: p-valor = 0.0569
 - Normalidad en niño: p-valor = 0.0972
 - Homocedasticidad (Levene): p-valor = 0.9452
 ANOVA clásico: p-valor = 0.3119

Variable: min_global_chl
 - Normalidad en niña: p-valor = 1.0000
 - Normalidad en niño: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
 Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niña: p-valor = 0.1193
 - Normalidad en niño: p-valor = 0.9265
 - Homocedasticidad (Levene): p-valor = 0.8332
 ANOVA clásico: p-valor = 0.1509

Variable: skew_global_chl
 - Normalidad en niña: p-valor = 0.6667
 - Normalidad en niño: p-valor = 0.4720
 - Homocedasti

Crear dataset estadísticos de Junio y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [5]:
dataset_estadisticos_jun_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "06"].reset_index(drop=True)

dataset_estadisticos_jun_eddy["fases"] = ["neutra", "neutra", "neutra", "neutra", "niña", "niña", "neutra",
                                      "niña", "neutra", "neutra", "neutra", "neutra", "niño", "neutra",
                                      "neutra", "neutra", "niño", "neutra", "neutra", "niña", "niño","neutra"]

dataset_estadisticos_jun_eddy = dataset_estadisticos_jun_eddy[dataset_estadisticos_jun_eddy["fases"] != "neutra"]
dataset_estadisticos_jun_eddy = dataset_estadisticos_jun_eddy.drop(columns = ["fases"])

X_estadisticos_jun_eddy = dataset_estadisticos_jun_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_jun_eddy

dataset_estadisticos_jun_eddy["fases"] = ["niña", "niña",
                                      "niña", "niño",
                                      "niño", "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_jun_eddy, col)


Variable: media_global_chl
 - Normalidad en niña: p-valor = 0.7282
 - Normalidad en niño: p-valor = 0.7133
 - Homocedasticidad (Levene): p-valor = 0.8792
ANOVA clásico: p-valor = 0.2119

Variable: std_global_chl
 - Normalidad en niña: p-valor = 0.6668
 - Normalidad en niño: p-valor = 0.3424
 - Homocedasticidad (Levene): p-valor = 0.9383
ANOVA clásico: p-valor = 0.3932

Variable: max_global_chl
 - Normalidad en niña: p-valor = 0.1892
 - Normalidad en niño: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = 0.0039
Welch: p-valor = 1.0000

Variable: min_global_chl
 - Normalidad en niña: p-valor = 1.0000
 - Normalidad en niño: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niña: p-valor = 0.7716
 - Normalidad en niño: p-valor = 0.8002
 - Homocedasticidad (Levene): p-valor = 0.7621
ANOVA clásico: p-valor = 0.2908

Variable: skew_global_chl
 - Normalidad en niña: p-valor = 0.7234
 - Normalidad en niño

Crear dataset estadísticos de Julio y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [6]:
dataset_estadisticos_jul_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "07"].reset_index(drop=True)

dataset_estadisticos_jul_eddy["fases"] = ["neutra", "niño", "neutra", "neutra", "niña", "neutra", "niño",
                                      "niña", "niña", "neutra", "neutra", "neutra", "niño", "neutra",
                                      "neutra", "neutra", "neutra", "neutra", "neutra", "niña", "niño","neutra"]

dataset_estadisticos_jul_eddy = dataset_estadisticos_jul_eddy[dataset_estadisticos_jul_eddy["fases"] != "neutra"]
dataset_estadisticos_jul_eddy = dataset_estadisticos_jul_eddy.drop(columns = ["fases"])

X_estadisticos_jul_eddy = dataset_estadisticos_jul_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_jul_eddy

dataset_estadisticos_jul_eddy["fases"] = ["niño", "niña", "niño",
                                      "niña", "niña", "niño",
                                      "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_jul_eddy, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.5954
 - Normalidad en niña: p-valor = 0.1545
 - Homocedasticidad (Levene): p-valor = 0.5748
ANOVA clásico: p-valor = 0.6252

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.1326
 - Normalidad en niña: p-valor = 0.7053
 - Homocedasticidad (Levene): p-valor = 0.9036
ANOVA clásico: p-valor = 0.6349

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0012
 - Normalidad en niña: p-valor = 0.0012
Mann-Whitney U: p-valor = 1.0000

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.2540
 - Normalidad en niña: p-valor = 0.0458
Mann-Whitney U: p-valor = 0.3429

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.5577
 - Normalidad en niña: p-valor = 0.8247
 - Homocedasticidad (Levene): p-valor = 0.1069
ANOVA clásico: p-v

Crear dataset estadísticos de Agosto y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [8]:
dataset_estadisticos_ago_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "08"].reset_index(drop=True)

dataset_estadisticos_ago_eddy["fases"] = ["niño", "neutra", "niño", "neutra", "neutra", "niña", "neutra", "niño",
                                      "niña", "niña", "neutra", "neutra", "neutra", "niño", "niña",
                                      "neutra", "neutra", "neutra", "niña", "niña", "niña", "niño","neutra"]

dataset_estadisticos_ago_eddy = dataset_estadisticos_ago_eddy[dataset_estadisticos_ago_eddy["fases"] != "neutra"]
dataset_estadisticos_ago_eddy = dataset_estadisticos_ago_eddy.drop(columns = ["fases"])

X_estadisticos_ago_eddy = dataset_estadisticos_ago_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_ago_eddy

dataset_estadisticos_ago_eddy["fases"] = ["niño", "niño", "niña", "niño",
                                      "niña", "niña", "niño", "niña",
                                      "niña", "niña", "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_ago_eddy, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.2290
 - Normalidad en niña: p-valor = 0.0079
Mann-Whitney U: p-valor = 1.0000

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.2441
 - Normalidad en niña: p-valor = 0.6950
 - Homocedasticidad (Levene): p-valor = 0.1908
ANOVA clásico: p-valor = 0.7792

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0001
 - Normalidad en niña: p-valor = 0.0491
Mann-Whitney U: p-valor = 0.3652

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.2444
 - Normalidad en niña: p-valor = 0.0146
Mann-Whitney U: p-valor = 1.0000

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.0817
 - Normalidad en niña: p-valor = 0.0260
Mann-Whitney U: p-valor = 1.0000

Variable: kurtosis_global_chl
 - Normalidad en niño: p-valor = 0.5265
 - Norm

Crear dataset estadísticos de Septiembre y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [12]:
dataset_estadisticos_sep_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "09"].reset_index(drop=True)

dataset_estadisticos_sep_eddy["fases"] = ["niño", "neutra", "niño", "neutra", "niño", "niña", "neutra", "niño",
                                      "niña", "niña", "neutra", "neutra", "neutra", "niño", "niña",
                                      "neutra", "niño", "neutra", "niña", "niña", "niña", "niño","neutra"]

dataset_estadisticos_sep_eddy = dataset_estadisticos_sep_eddy[dataset_estadisticos_sep_eddy["fases"] != "neutra"]
dataset_estadisticos_sep_eddy = dataset_estadisticos_sep_eddy.drop(columns = ["fases"])

X_estadisticos_sep_eddy = dataset_estadisticos_sep_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_sep_eddy

dataset_estadisticos_sep_eddy["fases"] = ["niño", "niño", "niño", "niña", "niño",
                                      "niña", "niña", "niño", "niña",
                                      "niño", "niña", "niña", "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_sep_eddy, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.2703
 - Normalidad en niña: p-valor = 0.1178
 - Homocedasticidad (Levene): p-valor = 0.2605
ANOVA clásico: p-valor = 0.5402

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.6827
 - Normalidad en niña: p-valor = 0.0280
Mann-Whitney U: p-valor = 0.8048

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0032
 - Normalidad en niña: p-valor = 0.0003 
Mann-Whitney U: p-valor = 0.7839

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.8847
 - Normalidad en niña: p-valor = 0.1523
 - Homocedasticidad (Levene): p-valor = 0.2881
ANOVA clásico: p-valor = 0.5170

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.2689
 - Normalidad en niña: p-valor = 0.4911
 - Homocedasticidad (Levene): p-valor = 0.4201
ANOVA clásico: p-

Crear dataset estadísticos de Octubre y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [10]:
dataset_estadisticos_oct_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "10"].reset_index(drop=True)

dataset_estadisticos_oct_eddy["fases"] = ["niño", "neutra", "niño", "neutra", "niño", "niña", "neutra", "niño",
                                      "niña", "niña", "neutra", "neutra", "niño", "niño", "niña",
                                      "niña", "niño", "neutra", "niña", "niña", "niña", "niño","neutra"]

dataset_estadisticos_oct_eddy = dataset_estadisticos_oct_eddy[dataset_estadisticos_oct_eddy["fases"] != "neutra"]
dataset_estadisticos_oct_eddy = dataset_estadisticos_oct_eddy.drop(columns = ["fases"])

X_estadisticos_oct_eddy = dataset_estadisticos_oct_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_oct_eddy

dataset_estadisticos_oct_eddy["fases"] = ["niño", "niño", "niño", "niña", "niño",
                                      "niña", "niña", "niño", "niño", "niña",
                                      "niña", "niño", "niña", "niña", "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_oct_eddy, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.3569
 - Normalidad en niña: p-valor = 0.1854
 - Homocedasticidad (Levene): p-valor = 0.0817
ANOVA clásico: p-valor = 0.2721

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.2647
 - Normalidad en niña: p-valor = 0.1907
 - Homocedasticidad (Levene): p-valor = 0.6833
ANOVA clásico: p-valor = 0.5326

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0006
 - Normalidad en niña: p-valor = 0.0064
Mann-Whitney U: p-valor = 0.1441

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.3889
 - Normalidad en niña: p-valor = 0.2340
 - Homocedasticidad (Levene): p-valor = 0.1842
ANOVA clásico: p-valor = 0.4907

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.4172
 - Normalidad en niña: p-valor = 0.2933
 - Homocedasticidad

Crear dataset estadísticos de Noviembre y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [3]:
dataset_estadisticos_nov_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "11"].reset_index(drop=True)

dataset_estadisticos_nov_eddy["fases"] = ["niño", "neutra", "niño", "niña", "niño", "niña",
                                      "niña", "niño", "niña", "niña", "neutra", "neutra",
                                      "niño", "niño", "niña", "niña", "niño", "neutra",
                                      "niña", "niña", "niña", "niño", "neutra"]

dataset_estadisticos_nov_eddy = dataset_estadisticos_nov_eddy[dataset_estadisticos_nov_eddy["fases"] != "neutra"]
dataset_estadisticos_nov_eddy = dataset_estadisticos_nov_eddy.drop(columns = ["fases"])

X_estadisticos_nov_eddy = dataset_estadisticos_nov_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_nov_eddy

dataset_estadisticos_nov_eddy["fases"] = ["niño", "niño", "niña", "niño", "niña",
                                      "niña", "niño", "niña", "niña",
                                      "niño", "niño", "niña", "niña", "niño",
                                      "niña", "niña", "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_nov_eddy, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.2680
 - Normalidad en niña: p-valor = 0.1605
 - Homocedasticidad (Levene): p-valor = 0.0311
Welch: p-valor = 0.0980

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.9354
 - Normalidad en niña: p-valor = 0.4982
 - Homocedasticidad (Levene): p-valor = 0.0660
ANOVA clásico: p-valor = 0.1173

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0005
 - Normalidad en niña: p-valor = 0.0009
Mann-Whitney U: p-valor = 0.3027

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.0454
 - Normalidad en niña: p-valor = 0.1407
Mann-Whitney U: p-valor = 0.3506

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.4206
 - Normalidad en niña: p-valor = 0.1600
 - Homocedasticidad (Levene): p-valor = 0.4930
ANOVA clásico: p-valor = 0

Crear dataset estadísticos de Diciembre y Contraste de hipótesis: Fase El Niño vs Fase La Niña zona eddy

In [11]:
dataset_estadisticos_oct_eddy = dataset_estadisticos_eddy[dataset_estadisticos_eddy["img_chl"].str[15:17] == "10"].reset_index(drop=True)

dataset_estadisticos_oct_eddy["fases"] = ["niño", "neutra", "niño", "niña", "niño", "niña",
                                      "niña", "niño", "niña", "niña", "neutra", "neutra",
                                      "niño", "niño", "niña", "niña", "niño", "neutra",
                                      "niña", "niña", "niña", "niño", "neutra"]

dataset_estadisticos_oct_eddy = dataset_estadisticos_oct_eddy[dataset_estadisticos_oct_eddy["fases"] != "neutra"]
dataset_estadisticos_oct_eddy = dataset_estadisticos_oct_eddy.drop(columns = ["fases"])

X_estadisticos_oct_eddy = dataset_estadisticos_oct_eddy.drop(columns = ["img_chl","img_poc","img_sst", "fecha"])
X_estadisticos_oct_eddy

dataset_estadisticos_oct_eddy["fases"] = ["niño", "niño", "niña", "niño", "niña",
                                      "niña", "niño", "niña", "niña",
                                      "niño", "niño", "niña", "niña", "niño",
                                      "niña", "niña", "niña", "niño"]


for col in ['media_global_chl', 'std_global_chl', 'max_global_chl',
       'min_global_chl', 'mediana_global_chl', 'skew_global_chl',
       'kurtosis_global_chl', 'media_izq_chl', 'media_cent_chl',
       'media_der_chl', 'std_izq_chl', 'std_cent_chl', 'std_der_chl',
       'media_global_poc', 'std_global_poc', 'max_global_poc',
       'min_global_poc', 'mediana_global_poc', 'skew_global_poc',
       'kurtosis_global_poc', 'media_izq_poc', 'media_cent_poc',
       'media_der_poc', 'std_izq_poc', 'std_cent_poc', 'std_der_poc',
       'media_global_sst', 'std_global_sst', 'max_global_sst',
       'min_global_sst', 'mediana_global_sst', 'skew_global_sst',
       'kurtosis_global_sst', 'media_izq_sst', 'media_cent_sst',
       'media_der_sst', 'std_izq_sst', 'std_cent_sst', 'std_der_sst']:

    analizar_contraste(dataset_estadisticos_oct_eddy, col)


Variable: media_global_chl
 - Normalidad en niño: p-valor = 0.3569
 - Normalidad en niña: p-valor = 0.0697
 - Homocedasticidad (Levene): p-valor = 0.0413
Welch: p-valor = 0.2059

Variable: std_global_chl
 - Normalidad en niño: p-valor = 0.2647
 - Normalidad en niña: p-valor = 0.3447
 - Homocedasticidad (Levene): p-valor = 0.7822
ANOVA clásico: p-valor = 0.6760

Variable: max_global_chl
 - Normalidad en niño: p-valor = 0.0006
 - Normalidad en niña: p-valor = 0.0017
Mann-Whitney U: p-valor = 0.1407

Variable: min_global_chl
 - Normalidad en niño: p-valor = 1.0000
 - Normalidad en niña: p-valor = 1.0000
 - Homocedasticidad (Levene): p-valor = nan
Welch: p-valor = 1.0000

Variable: mediana_global_chl
 - Normalidad en niño: p-valor = 0.3889
 - Normalidad en niña: p-valor = 0.1326
 - Homocedasticidad (Levene): p-valor = 0.1045
ANOVA clásico: p-valor = 0.4192

Variable: skew_global_chl
 - Normalidad en niño: p-valor = 0.4172
 - Normalidad en niña: p-valor = 0.2351
 - Homocedasticidad (Levene